In [ ]:
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

## Lecture 29

In [ ]:
def r_scatter(r):
    plots.figure(figsize=(5,5))
    "Generate a scatter plot with a correlation approximately r"
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x + (np.sqrt(1-r**2))*z
    plots.scatter(x, y)
    plots.xlim(-4, 4)
    plots.ylim(-4, 4)

In [ ]:
galton = Table.read_table('galton.csv')

In [ ]:
heights = Table().with_columns(
    'MidParent', galton.column('midparentHeight'),
    'Child', galton.column('childHeight')
    )
heights

In [ ]:
heights.scatter('MidParent')

In [ ]:
def predict_child(h):
    """Return a prediction of the height of a child 
    whose parents have a midparent height of h.
    
    The prediction is the average height of the children 
    whose midparent height is in the range h plus or minus 0.25 inches.
    """
    
    close_points = heights.where('MidParent', are.between(h-0.5, h + 0.5))
    return close_points.column('Child').mean()   

In [ ]:
heights_with_predictions = heights.with_column(
    'Prediction', heights.apply(predict_child, 'MidParent')
    )

In [ ]:
heights_with_predictions.scatter('MidParent')

In [ ]:
# data on hybrid passenger cars sold in US 1997 - 2013
#vehicle: model of the car
#year: year of manufacture
#msrp: manufacturer's suggested retail price in 2013 dollars
#acceleration: acceleration rate in km per hour per second
#mpg: fuel econonmy in miles per gallon
#class: the model's class.
hybrid = Table.read_table('hybrid.csv')
hybrid

In [ ]:
hybrid.sort('msrp', descending=True)

In [ ]:
#vehicles with higher mpg tend to cost less on average - surprising?
hybrid.scatter('mpg', 'msrp')

In [ ]:
#vehicles that accelerate faster tend to cost more, and have lower mpg (not as fuel efficient)
hybrid.scatter('acceleration', 'msrp')

In [ ]:
#msrp vs mpg has curved negative association; let's restrict to suv's
suv = hybrid.where('class', 'SUV')
suv.num_rows

In [ ]:
suv.scatter('acceleration', 'msrp')

In [ ]:
suv.scatter('mpg', 'msrp')

In [ ]:
#Convert data to standard units
def standard_units(x):
    "Convert any array of numbers to standard units."
    return (x - np.average(x))/np.std(x)  

In [ ]:
suv = suv.with_columns(
    'mpg (standard units)',  standard_units(suv.column('mpg')), 
    'msrp (standard units)', standard_units(suv.column('msrp')),
    'acceleration (standard units)', standard_units(suv.column('acceleration')), 
)
suv

In [ ]:
#Standard units does not change the point patterns
suv.scatter('mpg (standard units)', 'msrp (standard units)')
plots.xlim(-3, 3)
plots.ylim(-3, 3);

In [ ]:
#Comparing the scatterplot above and below (easier now that they are in standard units)
### we can wee the msrp(su) vs mpg(su) has less scatter around linear relationship
# How to quantify this? -> Correlation coefficient!
suv.scatter('acceleration (standard units)', 'msrp (standard units)')
plots.xlim(-3, 3)
plots.ylim(-3, 3);

### Correlation coefficient

In [ ]:
r_scatter(0.3)

In [ ]:
r_scatter(0)

In [ ]:
r_scatter(-0.2)

In [ ]:
r_scatter(-.95)

In [ ]:
r_scatter(-.7)

### Calculating the correlation coefficient

In [ ]:
x = np.arange(1, 7, 1)
y = make_array(2, 3, 1, 5, 2, 7)
t = Table().with_columns(
        'x', x,
        'y', y
    )
t

In [ ]:
t.scatter('x', 'y', s=30, color='red')

In [ ]:
t = t.with_columns(
        'x (standard units)', standard_units(x),
        'y (standard units)', standard_units(y)
    )
t

In [ ]:
t.scatter('x (standard units)', 'y (standard units)', s=30, color='red')
plots.xlim(-2.5, 2.5)
plots.ylim(-2.5, 2.5);

In [ ]:
x_std_units = t.column('x (standard units)')
y_std_units = t.column('y (standard units)')
t = t.with_column(
    'product of standard units', x_std_units * y_std_units)
t

In [ ]:
# r is the average of the products of standard units
r = np.mean(t.column('product of standard units'))
r

In [ ]:
def correlation(t, label_x, label_y):
    x_in_standard_units = standard_units(t.column(label_x))
    y_in_standard_units = standard_units(t.column(label_y))
    return np.mean(x_in_standard_units * y_in_standard_units)

In [ ]:
correlation(t, "x", "y")

In [ ]:
correlation(suv, 'mpg', 'msrp')

In [ ]:
correlation(suv, 'acceleration', 'msrp')

In [ ]:
correlation(t, 'x', 'y')

In [ ]:
#Order doesn't matter
correlation(t, 'y', 'x')

In [ ]:
t.scatter('x', 'y')

In [ ]:
t.scatter('y', 'x')

In [ ]:
#Pattern, but nonlinear
new_x = np.arange(-4, 4.1, 0.5)
nonlinear = Table().with_columns(
        'x', new_x,
        'y', new_x**2
    )
nonlinear.scatter('x', 'y', s=30, color='r')

In [ ]:
correlation(nonlinear, 'x', 'y')

In [ ]:
line = Table().with_columns(
        'x', make_array(1, 2, 3, 4),
        'y', make_array(1, 2, 3, 4)
    )
line.scatter('x', 'y', s=30, color='r')

In [ ]:
correlation(line, 'x', 'y')

In [ ]:
outlier = Table().with_columns(
        'x', make_array(1, 2, 3, 4, 5),
        'y', make_array(1, 2, 3, 4, 0)
    )
outlier.scatter('x', 'y', s=30, color='r')

In [ ]:
correlation(outlier, 'x', 'y')

In [ ]:
#Ecological correlations: correlations using data that are aggregated
by_state = Table.read_table('election2016_state.csv')
by_state

In [ ]:
#Positive association between median income (of state) and percent voting for Clinton
by_state.scatter('Median Income', 'Percent voting for Clinton')

In [ ]:
party = Table.read_table('party_affiliation2016.csv')
party

In [ ]:
party.barh('Income')

In [ ]:
#participation rate=% of high school seniors who took the test
sat2014 = Table.read_table('sat2014.csv').sort('State')
sat2014

In [ ]:
sat2014.scatter('Critical Reading', 'Math')

In [ ]:
# Not the strength of the relation between Math and Critial Reading scores *students*
# All students are lumpted together by state
# Plotting this at the student level will display more variability (lower correlation)
correlation(sat2014, 'Critical Reading', 'Math')

In [ ]:
by_district = Table.read_table('election2016_district.csv')
by_district

In [ ]:
by_district.scatter('Median Income', 'Percent voting for Clinton')

In [ ]:
correlation(by_district, 'Median Income', 'Percent voting for Clinton')